# Classes and types in python

Python is actually a complicated language when it comes to classes.  It has true multiple inheritance, unlike Java where
a class may only extend one class, but implement many interfaces. Not only does it support true multiple inheritance,
but it also has a different form of subtyping than traditional OOP languages. Which then begs a couple of questions and
uses of other languages sometimes get confused about how to implement OOP in python.

So, to help understand all this, this lesson will cover the basics of python classes and its type system

- What is `object`
- Basic class definition
- What is a `@dataclass`
- How does multiple inheritance work?
- What is the equivalent of an `interface` in python then?
